# Summary Stats

In [1]:
import sqlite3

import httpx
import pandas as pd

In [2]:
pd.set_option("display.float_format", lambda x: f"{x:0.1f}")

## API Endpoints

In [3]:
API_BASE = "https://clinicaltrials.gov/api/v2"
API_FIELD_VALUES = API_BASE + "/stats/field/values"

## Retrieve Integer/Number Fields

In [4]:
response = httpx.get(API_FIELD_VALUES, params={"types": "NUMBER|INTEGER"})
response.raise_for_status()
data = response.json()

In [5]:
data

[{'type': 'INTEGER',
  'piece': 'DispFirstPostYear',
  'field': 'protocolSection.statusModule.dispFirstPostDateStruct.dispFirstPostYear',
  'missingStudiesCount': 491584,
  'min': 2009,
  'max': 2024,
  'avg': 2015.9466914585655},
 {'type': 'INTEGER',
  'piece': 'DispFirstSubmitYear',
  'field': 'protocolSection.statusModule.dispFirstSubmitYear',
  'missingStudiesCount': 491541,
  'min': 2008,
  'max': 2024,
  'avg': 2015.7891834561785},
 {'type': 'INTEGER',
  'piece': 'EnrollmentCount',
  'field': 'protocolSection.designModule.enrollmentInfo.count',
  'missingStudiesCount': 6919,
  'min': 0,
  'max': 188814085,
  'avg': 5350.113494120964},
 {'type': 'INTEGER',
  'piece': 'EventGroupDeathsNumAffected',
  'field': 'resultsSection.adverseEventsModule.eventGroups.deathsNumAffected',
  'missingStudiesCount': 465825,
  'min': 0,
  'max': 17251,
  'avg': 7.102420993826784},
 {'type': 'INTEGER',
  'piece': 'EventGroupDeathsNumAtRisk',
  'field': 'resultsSection.adverseEventsModule.eventGroups